1. Start a simple Spark session

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://mirrors.sonic.net/apache/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
!tar xzf spark-3.1.2-bin-hadoop3.2.tgz
!pip install -q findspark


import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"


import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
spark

2. Load the Walmart Stock CSV file, let Spark infer the data types

In [ ]:
df = spark.read.options(header='True', inferSchema='True').csv('drive/MyDrive/Colab Notebooks/walmart_stock.csv')
df.registerTempTable('data')

3. Show the column names

In [ ]:
df.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

4. What does the Schema look like?

In [ ]:
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



5. Print out the first 5 rows

In [ ]:
df.show(5)

+----------+------------------+---------+---------+------------------+--------+------------------+
|      Date|              Open|     High|      Low|             Close|  Volume|         Adj Close|
+----------+------------------+---------+---------+------------------+--------+------------------+
|2012-01-03|         59.970001|61.060001|59.869999|         60.330002|12668800|52.619234999999996|
|2012-01-04|60.209998999999996|60.349998|59.470001|59.709998999999996| 9593300|         52.078475|
|2012-01-05|         59.349998|59.619999|58.369999|         59.419998|12768200|         51.825539|
|2012-01-06|         59.419998|59.450001|58.869999|              59.0| 8069400|          51.45922|
|2012-01-09|         59.029999|59.549999|58.919998|             59.18| 6679300|51.616215000000004|
+----------+------------------+---------+---------+------------------+--------+------------------+
only showing top 5 rows



In [ ]:
spark.sql('SELECT * FROM data LIMIT 5').toPandas()

,Date,Open,High,Low,Close,Volume,Adj Close
0,2012-01-03,59.970001,61.060001,59.869999,60.330002,12668800,52.619235
1,2012-01-04,60.209999,60.349998,59.470001,59.709999,9593300,52.078475
2,2012-01-05,59.349998,59.619999,58.369999,59.419998,12768200,51.825539
3,2012-01-06,59.419998,59.450001,58.869999,59.000000,8069400,51.459220
4,2012-01-09,59.029999,59.549999,58.919998,59.180000,6679300,51.616215


6. Use describe() to learn about the DataFrame

In [ ]:
df.describe().toPandas()

,summary,Date,Open,High,Low,Close,Volume,Adj Close
0,count,1258,1258,1258,1258,1258,1258,1258
1,mean,None,72.35785375357709,72.83938807631165,71.9186009594594,72.38844998012726,8222093.481717011,67.23883848728146
2,stddev,None,6.76809024470826,6.768186808159218,6.744075756255496,6.756859163732991,4519780.8431556,6.722609449996857
3,min,2012-01-03,56.389998999999996,57.060001,56.299999,56.419998,2094900,50.363689
4,max,2016-12-30,90.800003,90.970001,89.25,90.470001,80898100,84.91421600000001


7. (optional and not easy) Format the numbers in the describe() table to show only 2 decimal places

In [ ]:
from pyspark.sql.functions import format_number
desc = df.describe()
desc.select('summary', 'Date', 
            format_number(desc['Open'].cast('float'), 2).alias('Open'),
            format_number(desc['High'].cast('float'), 2).alias('High'),
            format_number(desc['Low'].cast('float'), 2).alias('Low'),
            format_number(desc['Close'].cast('float'), 2).alias('Close'),
            format_number(desc['Volume'].cast('float'), 2).alias('Volume'),
            format_number(desc['Adj Close'].cast('float'), 2).alias('Adj Close')
           ).show()

+-------+----------+--------+--------+--------+--------+-------------+---------+
|summary|      Date|    Open|    High|     Low|   Close|       Volume|Adj Close|
+-------+----------+--------+--------+--------+--------+-------------+---------+
|  count|      1258|1,258.00|1,258.00|1,258.00|1,258.00|     1,258.00| 1,258.00|
|   mean|      null|   72.36|   72.84|   71.92|   72.39| 8,222,093.50|    67.24|
| stddev|      null|    6.77|    6.77|    6.74|    6.76| 4,519,781.00|     6.72|
|    min|2012-01-03|   56.39|   57.06|   56.30|   56.42| 2,094,900.00|    50.36|
|    max|2016-12-30|   90.80|   90.97|   89.25|   90.47|80,898,096.00|    84.91|
+-------+----------+--------+--------+--------+--------+-------------+---------+



8. Create a new DataFrame with a column called  'HV Ratio' that is the ratio of the High Price vs Volume of Stock traded for a day

In [ ]:
df2 = df.withColumn('HV Ratio', df['High'] / df['Volume'])
df2.select('HV Ratio').show()

+--------------------+
|            HV Ratio|
+--------------------+
|4.819714653321546E-6|
|6.290848613094555E-6|
|4.669412994783916E-6|
|7.367338463826307E-6|
|8.915604778943901E-6|
|8.644477436914568E-6|
|9.351828421515645E-6|
| 8.29141562102703E-6|
|7.712212102001476E-6|
|7.071764823529412E-6|
|1.015495466386981E-5|
|6.576354146362592...|
| 5.90145296180676E-6|
|8.547679455011844E-6|
|8.420709512685392E-6|
|1.041448341728929...|
|8.316075414862431E-6|
|9.721183814992126E-6|
|8.029436027707578E-6|
|6.307432259386365E-6|
+--------------------+
only showing top 20 rows



In [ ]:
spark.sql('SELECT (High / Volume) AS `HV Ratio` FROM data').show()

+--------------------+
|            HV Ratio|
+--------------------+
|4.819714653321546E-6|
|6.290848613094555E-6|
|4.669412994783916E-6|
|7.367338463826307E-6|
|8.915604778943901E-6|
|8.644477436914568E-6|
|9.351828421515645E-6|
| 8.29141562102703E-6|
|7.712212102001476E-6|
|7.071764823529412E-6|
|1.015495466386981E-5|
|6.576354146362592...|
| 5.90145296180676E-6|
|8.547679455011844E-6|
|8.420709512685392E-6|
|1.041448341728929...|
|8.316075414862431E-6|
|9.721183814992126E-6|
|8.029436027707578E-6|
|6.307432259386365E-6|
+--------------------+
only showing top 20 rows



9. What day had the Peak High in Price?

In [ ]:
df.orderBy(df['High'].desc()).head(1)[0][0]

'2015-01-13'

In [ ]:
spark.sql('SELECT Date FROM data ORDER BY High DESC LIMIT 1').show()

+----------+
|      Date|
+----------+
|2015-01-13|
+----------+



10. What is the mean of the Close column

In [ ]:
from pyspark.sql.functions import mean
df.select(mean(df['Close'])).show()

+-----------------+
|       avg(Close)|
+-----------------+
|72.38844998012726|
+-----------------+



In [ ]:
spark.sql('SELECT MEAN(Close) FROM data').show()

+-----------------+
|      mean(Close)|
+-----------------+
|72.38844998012726|
+-----------------+



11. What is the max and min of the Volume column?

In [ ]:
df.groupBy().agg({'Volume': 'max'}).show()

+-----------+
|max(Volume)|
+-----------+
|   80898100|
+-----------+



In [ ]:
df.groupBy().max('Volume').show()
df.groupBy().min('Volume').show()

+-----------+
|max(Volume)|
+-----------+
|   80898100|
+-----------+

+-----------+
|min(Volume)|
+-----------+
|    2094900|
+-----------+



In [ ]:
spark.sql('SELECT MAX(Volume),MIN(Volume) FROM data').show()

+-----------+-----------+
|max(Volume)|min(Volume)|
+-----------+-----------+
|   80898100|    2094900|
+-----------+-----------+



12. How many days was the Close lower than 60 dollars?

In [ ]:
df.filter(df['Close'] < 60).count()

81

In [ ]:
spark.sql('SELECT * FROM data WHERE Close < 60').count()

81

In [ ]:
spark.sql('SELECT COUNT(CASE WHEN Close < 60 THEN 1 END) AS `Close < 60` FROM data').show()

+----------+
|Close < 60|
+----------+
|        81|
+----------+



13. What percentage of time was the High greater than 80 dollars?

In [ ]:
df.filter(df['High'] > 80).count() / df.count() * 100

9.141494435612083

In [ ]:
spark.sql('SELECT COUNT(CASE WHEN High > 80 THEN 1 END) * 100 / COUNT(*) AS `Percentage High > 80` \
           FROM data').show()

+--------------------+
|Percentage High > 80|
+--------------------+
|   9.141494435612083|
+--------------------+



14. What is the Pearson correlation between High and Volume?

In [ ]:
df.corr('High', 'Volume')

-0.3384326061737161

In [ ]:
spark.sql('SELECT corr(High, Volume) AS `corr(High,Volume)` FROM data').show()

+-------------------+
|  corr(High,Volume)|
+-------------------+
|-0.3384326061737161|
+-------------------+



15. What is the max High per year?

In [ ]:
from pyspark.sql.functions import year
df2 = df.withColumn('Year', year(df['Date']))
df2.groupBy('Year').max('High').orderBy('Year').show()

+----+---------+
|Year|max(High)|
+----+---------+
|2012|77.599998|
|2013|81.370003|
|2014|88.089996|
|2015|90.970001|
|2016|75.190002|
+----+---------+



In [ ]:
spark.sql('SELECT year(Date) AS Year, max(High) \
           FROM data \
           GROUP BY year(Date) \
           ORDER BY year(Date)').show()

+----+---------+
|Year|max(High)|
+----+---------+
|2012|77.599998|
|2013|81.370003|
|2014|88.089996|
|2015|90.970001|
|2016|75.190002|
+----+---------+



16. What is the average Close for each calendar month?

In [ ]:
from pyspark.sql.functions import month
df2 = df.withColumn('Month', month(df['Date']))
df2.groupBy('Month').mean('Close').orderBy('Month').show()

+-----+-----------------+
|Month|       avg(Close)|
+-----+-----------------+
|    1|71.44801958415842|
|    2|  71.306804443299|
|    3|71.77794377570092|
|    4|72.97361900952382|
|    5|72.30971688679247|
|    6| 72.4953774245283|
|    7|74.43971943925233|
|    8|73.02981855454546|
|    9|72.18411785294116|
|   10|71.57854545454543|
|   11| 72.1110893069307|
|   12|72.84792478301885|
+-----+-----------------+



In [ ]:
spark.sql('SELECT month(Date) AS Month, mean(Close) \
           FROM data \
           GROUP BY month(Date) \
           ORDER BY month(Date)').show()

+-----+-----------------+
|Month|      mean(Close)|
+-----+-----------------+
|    1|71.44801958415842|
|    2|  71.306804443299|
|    3|71.77794377570092|
|    4|72.97361900952382|
|    5|72.30971688679247|
|    6| 72.4953774245283|
|    7|74.43971943925233|
|    8|73.02981855454546|
|    9|72.18411785294116|
|   10|71.57854545454543|
|   11| 72.1110893069307|
|   12|72.84792478301885|
+-----+-----------------+



In [ ]:
spark.sql('SELECT concat(year(Date),"-",month(Date)) AS YearMonth, mean(Close) \
           FROM data GROUP BY YearMonth \
           ORDER BY YearMonth').show()

+---------+------------------+
|YearMonth|       mean(Close)|
+---------+------------------+
|   2012-1|        60.2354999|
|  2012-10| 75.30619061904761|
|  2012-11| 71.10952333333333|
|  2012-12| 69.71100009999999|
|   2012-2|            60.898|
|   2012-3|60.433636818181796|
|   2012-4|60.149000150000006|
|   2012-5|61.456363409090905|
|   2012-6| 67.50380961904762|
|   2012-7| 72.40666661904763|
|   2012-8| 73.04478265217392|
|   2012-9| 74.18157921052631|
|   2013-1| 69.09476142857143|
|  2013-10| 74.97913104347826|
|  2013-11| 78.97300075000001|
|  2013-12|  78.7752382857143|
|   2013-2| 70.62315857894738|
|   2013-3| 73.43649940000002|
|   2013-4| 77.68954572727273|
|   2013-5| 77.81636368181817|
+---------+------------------+
only showing top 20 rows

